<a href="https://colab.research.google.com/github/datasistah/STAR_method/blob/main/StarMethodV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U git+https://github.com/lvwerra/trl.git git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git git+https://github.com/huggingface/peft.git
!pip install -q -U datasets bitsandbytes einops wandb
!pip install rouge-score


!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers


  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.1 MB/s et

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://vilsonrodrigues.medium.com/run-your-private-llm-falcon-7b-instruct-with-less-than-6gb-of-gpu-using-4-bit-quantization-ff1d4ffbabcc


#### Imports

In [ ]:
#CUDA error message
#!nvcc --version

In [ ]:
# !git clone https://github.com/TimDettmers/bitsandbytes.git
# %cd bitsandbytes
# !CUDA_VERSION=118 make cuda11x
# !python setup.py install


In [3]:
#CUDA error message
import os
os.environ["CUDA_VERSION"] = "118"
os.environ["LD_LIBRARY_PATH"] += ":/usr/local/cuda/lib64"


In [4]:
import torch, einops
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft.tuners.lora import LoraLayer

#### Load Quantized Model



In [5]:
#model_id = "ybelkada/falcon-7b-sharded-bf16"
model_id = "tiiuae/falcon-7b"
compute_dtype = getattr(torch, "float16")

#special config for bitsandbytes that enables 4bit quantization
bnb_config = BitsAndBytesConfig(
              load_in_4bit=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=compute_dtype,
              bnb_4bit_use_double_quant=True,
              )

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
                            model_id,
                            quantization_config=bnb_config,
                            trust_remote_code=True,
                            )


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.9,
    repetition_penalty=1.2,
    max_new_tokens=512,
    device_map="auto",
    do_sample=True
)



In [7]:
pipe("As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements. Return as valid json. ")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': 'As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements. Return as valid json. { "interview": "STAR", "interviewer": [ "HR Manager", "HR Manager", "HR Manager", "HR Manager" ], "interviewee": [ { "name": "Bill R", "position": "Customer Service Manager", "question": "I am looking for a hard working, driven person who is reliable to work on the night shift in our call center." }, { "name": "Gus R", "position": "Customer Service Representative", "question": "We are looking for someone who is self motivated and has a desire to grow in our customer service department. You will be taking orders over the phone and processing requests for refunds and exchange." } ], "question": "What is the difference between someone who is good and someone who is great?" }'}]

In [8]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [9]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
#test PromptTemplate
# from langchain. prompts import PromptTemplate
# multi_prompt= PromptTemplate(input_variables=["adjective", "content"],
#                        template="suggest a good name for the {content} with the genre {adjective}",)

# print(multi_prompt.format(adjective="fictional", content="book"))

In [10]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipe)

prompt = PromptTemplate(
      input_variables=['question', 'answer'],
      template="""
   As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements.
   question: {question}, answer: {answer}
   Return your response in json format.
    """
    )

question1 = "Tell me about a time when you received criticism that you thought was unfair."
answer1 = "In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions and this improved the speed of my call handling rates, meaning I could help and assist more customers."
#test out prompt
print(prompt.format(question=question1, answer = answer1))



   As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements.
   question: Tell me about a time when you received criticism that you thought was unfair., answer: In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions an

In [11]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question1, answer = answer1)
response

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'{\n      "user": "mikejacks",\n      "comment": true\n    }\n  \n## 2.3.3 The "Why" interview question\n **Task:** You have a choice of two candidates for a job: Candidate A, or Candidate B. You must choose Candidate A.\n\n \n ``` \nSELECT candidate_id, resume_url\n  FROM candidates AS c1 JOIN jobs AS j1\n    ON j1.candidate_id = c1.candidate_id;\n  SELECT job_title\n  FROM jobs;\n```\n\n**What**: In this sample job, you must choose either Candidate A or Candidate B for the same job. Candidate A has more experience and better ratings from colleagues than Candidate B.\n**Task**: You must select Candidate A.\n**Why**: We have shown that the candidate\'s experience is higher and their ratings are greater than those for Candidate B. We can only choose Candidate A because they have a better level of performance. \n\n**Return a JSON object which should contain information in the following format.**\n\n``` json\n{\n  "candidateA": {\n    "candidate_id": 1,\n    "resume_url": "http://mycompan

#### Prepare Model for Training


*   activate gradient_checkpointing which significally reduces memory
*   print trainable parameters


> https://huggingface.co/docs/peft/task_guides/semantic_segmentation_lora






In [12]:
from peft import prepare_model_for_kbit_training


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [13]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

#### Create LoRa Config and Appply to Model


*   This is when we add the low ranking adapters/parameters to the model which are the only parameters that will be trained.

*   Increasing the "r" parameter in LoraConfig will add more adapters to the model.



In [16]:
 from peft import LoraConfig, get_peft_model

 peft_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        bias="lora_only",
        task_type="CAUSAL_LM",
        modules_to_save=['decode_head'],
        target_modules=[
          "query",
          "value"
        ]
    )

 model = get_peft_model(model, peft_config)
 print_trainable_parameters(model)

trainable params: 2359296 || all params: 3611104128 || trainable%: 0.06533447711203746


#### Prepare Data


> https://huggingface.co/docs/datasets/v1.1.3/loading_datasets.html



In [17]:
pwd

'/content'

In [18]:
!ls

drive  sample_data


In [19]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/StarMethod/Data/STAR_Method_Data.xlsx', sheet_name='Sheet2')
df.head()

,url,Question,Answer,STAR Components
0,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you received critici...,"In a previous role, my manager spoke to me abo...",Situation: My manager criticized the length of...
1,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you had to do someth...,I was working in a previous role as part of a ...,Situation: Team member's illness created a gap...
2,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you worked in a team.,"In a previous role, I was part of a team whose...",Situation: Team responsible for promoting a ne...
3,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you made a mistake.,"When I started working my last role, I was eag...",Situation: Overcommitted to tasks and missed a...
4,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you multitasked.,My manager asked for a volunteer to cover the ...,Situation: Volunteered to cover the work of a ...


In [22]:
#df['STAR Components'][188]


'Missing Components: Task, Action, Result <br> Suggestion: Provide more context about the specific task, your actions, and the result. <br> Improved Answer: When I was working as an IT manager, we experienced a major system failure that halted our operations. My task was to identify the issue and restore the system as quickly as possible. I led my team in troubleshooting the problem, and we worked tirelessly to get the system back up. As a result, we were able to minimize the downtime and resume operations within a few hours.'

In [23]:
#df['STAR Components'][188].replace("<br>", '')
df['output'] = df['STAR Components'].apply(lambda x: x.replace("<br>", ''))
df['output'][188]

'Missing Components: Task, Action, Result  Suggestion: Provide more context about the specific task, your actions, and the result.  Improved Answer: When I was working as an IT manager, we experienced a major system failure that halted our operations. My task was to identify the issue and restore the system as quickly as possible. I led my team in troubleshooting the problem, and we worked tirelessly to get the system back up. As a result, we were able to minimize the downtime and resume operations within a few hours.'

In [24]:
df['question'] = df['Question'].apply(lambda x: "question:" + x)
df['answer'] = df['Answer'].apply(lambda x: "answer:" + x)

df.head()

,url,Question,Answer,STAR Components,output,question,answer
0,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you received critici...,"In a previous role, my manager spoke to me abo...",Situation: My manager criticized the length of...,Situation: My manager criticized the length of...,question:Tell me about a time when you receive...,"answer:In a previous role, my manager spoke to..."
1,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you had to do someth...,I was working in a previous role as part of a ...,Situation: Team member's illness created a gap...,Situation: Team member's illness created a gap...,question:Tell me about a time when you had to ...,answer:I was working in a previous role as par...
2,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you worked in a team.,"In a previous role, I was part of a team whose...",Situation: Team responsible for promoting a ne...,Situation: Team responsible for promoting a ne...,question:Tell me about a time when you worked ...,"answer:In a previous role, I was part of a tea..."
3,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you made a mistake.,"When I started working my last role, I was eag...",Situation: Overcommitted to tasks and missed a...,Situation: Overcommitted to tasks and missed a...,question:Tell me about a time when you made a ...,"answer:When I started working my last role, I ..."
4,https://www.youtube.com/watch?v=2uM7gYuOvr4,Tell me about a time when you multitasked.,My manager asked for a volunteer to cover the ...,Situation: Volunteered to cover the work of a ...,Situation: Volunteered to cover the work of a ...,question:Tell me about a time when you multita...,answer:My manager asked for a volunteer to cov...


In [25]:
df["input"] = df[["question", "answer"]].apply(",".join, axis=1)
df['input'][186]

'question:Can you describe a time when you had to deal with a difficult customer?,answer:I dealt with a customer who was unhappy with our product.'

In [26]:
data= df[['input','output']]
data.head()

,input,output
0,question:Tell me about a time when you receive...,Situation: My manager criticized the length of...
1,question:Tell me about a time when you had to ...,Situation: Team member's illness created a gap...
2,question:Tell me about a time when you worked ...,Situation: Team responsible for promoting a ne...
3,question:Tell me about a time when you made a ...,Situation: Overcommitted to tasks and missed a...
4,question:Tell me about a time when you multita...,Situation: Volunteered to cover the work of a ...


In [27]:
data = data.sample(frac=1).reset_index(drop=True)
data.tail()

,input,output
186,"question:How do you handle a challenge?,answer...","Missing Components: Situation, Task, Action, R..."
187,question:Tell me about a time when you had to ...,Situation: As a Customer Service Representativ...
188,question:Can you describe a time when you had ...,"Missing Components: Situation, Task, Action, R..."
189,question:Can you tell me about a time when you...,Situation: Struggling to increase engagement o...
190,question:Can you give me an example of a time ...,Situation: Finding new business opportunities ...


#### Example Before Fine Tuning

*   using data without training
*   convert text to pytorch tensors



In [28]:
pwd

'/content'

In [29]:
output_dir = "/drive/MyDrive/StarMethod/"

In [31]:
data.to_json('/content/drive/MyDrive/StarMethod/Data/StarMethod_training_data.jsonl', lines=True, orient='records')

#### Training


*   TRL- HuggingFace "Transformer Reinforcement Learning" library
*   SFTTrainer- "supervised fine-tuning"
*   "paged_adamw_32bit" is powerful optimizer, helps to avoid memory overload



In [32]:
len(data)

191

In [33]:
type(data)

pandas.core.frame.DataFrame

In [34]:
from datasets import Dataset
temp_df = pd.DataFrame({"a": [1, 2, 3]})
temp_df

,a
0,1
1,2
2,3


In [35]:
temp_dataset = Dataset.from_pandas(temp_df)
temp_dataset

Dataset({
    features: ['a'],
    num_rows: 3
})

In [36]:
data.head()

,input,output
0,question:Tell me about a time you worked under...,Situation: My manager asked for a volunteer to...
1,question:Can you tell me about a time when you...,"Missing Components: Situation, Task, Action, R..."
2,question:How do you handle stress and pressure...,"Missing Components: Situation, Task, Action, R..."
3,question:Tell me about a time when you had to ...,"Missing Components: Situation, Task, Action, R..."
4,question:Can you give me an example of a time ...,Situation: Negotiating with suppliers for bett...


In [56]:
#convert pandas df with data to a Dataset format for training
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 191
})

In [63]:
dataset[0]['input']

"question:Tell me about a time you worked under pressure,answer:My manager asked for a volunteer to complete the work of a co-worker who had recently left the company. A replacement team member had yet to be found so they needed someone to do their work in the interim. I saw this as an opportunity to diversify my skills so I volunteered. It was my task to learn my former co-workers duties quickly and complete their work and my own on time. I studied their job description and ticked off the work I could quickly complete. Now for the tasks I was unsure of, I read the company operating procedures and used online training tutorials. Now once confidence I prioritized all my tasks based on the company's needs. Red tasks were urgent and came first, Amber tasks second and green tasks could be completed when I had more time available. Now by using a systematic approach to learning and completing all tasks, I could do the work of two people which helped the company in its time of need."

In [66]:
#import supervised fine-tuning from HF library
#setup the trainer

from trl import SFTTrainer

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=30,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=500,
    warmup_ratio=0.03
    )

In [65]:
def formatting_prompts_func(example):
    text = f"### input: {example['input']}\n ### output: {example['output']}"
    return text

In [67]:
#format dataset by combining input/output column
formatting_prompts_func(dataset[2])

'### input: question:How do you handle stress and pressure?,answer:I take a break and do some deep breathing.\n ### output: Missing Components: Situation, Task, Action, Result  Suggestion: Share a specific instance where you were under stress, how you handled it, and the outcome.  Improved Answer: During a high-stakes project at my last job, I was under significant stress due to tight deadlines. I managed this by taking short breaks for deep breathing exercises, which helped me maintain my focus. As a result, I was able to effectively manage the project and meet the deadline.'

In [68]:
len(dataset)

191

In [69]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=512,
    formatting_func=formatting_prompts_func,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)
model.config.use_cache = False  # silence the warnings
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.818000
2,1.816300
3,1.930500
4,1.737300
5,1.902400
6,1.824500
7,1.842900
8,1.853400
9,1.832000
10,1.798800


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7d9c95bf38e0> was reported to be 191 (when accessing len(dataloader)), but 192 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7d9c95bf38e0> was reported to be 191 (when accessing len(dataloader)), but 193 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:645: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7d9c

TrainOutput(global_step=500, training_loss=0.8258352880477905, metrics={'train_runtime': 1568.1404, 'train_samples_per_second': 5.102, 'train_steps_per_second': 0.319, 'total_flos': 8.1485047529472e+16, 'train_loss': 0.8258352880477905, 'epoch': 41.67})

#### Save Model


*   local directory
*   HuggingFace



In [70]:
#Save to directory
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)

In [74]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [75]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

In [73]:
!git config --global credential.helper store

In [76]:
#push model to HuggingFace Hub
model_to_save.push_to_hub(repo_id='datasistah/STARMethod_Interview_Model')

adapter_model.bin:   0%|          | 0.00/9.46M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasistah/STARMethod_Interview_Model/commit/c636a27a964491ba192b7bde8d16373184b7127d', commit_message='Upload model', commit_description='', oid='c636a27a964491ba192b7bde8d16373184b7127d', pr_url=None, pr_revision=None, pr_num=None)

#### Import Model for Inference

In [80]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("datasistah/STARMethod_Interview_Model")
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)
model = PeftModel.from_pretrained(model, "datasistah/STARMethod_Interview_Model")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)

### Inference after training

In [82]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipe)

prompt = PromptTemplate(
      input_variables=['question', 'answer'],
      template="""
   As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements.
   question: {question}, answer: {answer}
   Return your response in json format.
    """
    )

question1 = "Tell me about a time when you received criticism that you thought was unfair."
answer1 = "In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions and this improved the speed of my call handling rates, meaning I could help and assist more customers."
#test out prompt
print(prompt.format(question=question1, answer = answer1))


   As a career coach, analyze the following interview response using the STAR method. If the method is used correctly, provide a summary of each component. If not, identify the missing components and suggest improvements.
   question: Tell me about a time when you received criticism that you thought was unfair., answer: In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions an

In [84]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question1, answer = answer1)
response

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


KeyboardInterrupt: ignored

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question="Tell me about a time when you received criticism that you thought was unfair.", answer = "In a previous role, my manager spoke to me about how long I was speaking to customers for on the telephone. I was trying to build up a positive rapport with our customers to help improve sales. My manager said I was taking too long. Although I felt the criticism was unfair because I was trying to help the company, it was my task to take on board his feedback to improve. I explained to my manager that I was trying to improve customer relations but that I fully understood there was a need to act with speed because other customers were waiting on the line that also needed my assistance. After reflecting on my performance, I made changes as per my manager's instructions and this improved the speed of my call handling rates, meaning I could help and assist more customers.")
response

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


"> Lorem Ipsum is used as a filler text for a form template. <br> Good Answer <br> Your answer should be at least one paragraph long. Where's the problem? What actions did you take? And how did you manage to solve the issue? Lorem Ipsum"

In [ ]:
question3 = "What is your greatest achievement?"
answer3 = "When I was a manager at Cafe X, we would always get a lot more customers than usual during the summer. It would almost always be packed, and sometimes, there were not enough servers to handle the work. So, I had to come up with a system so that we would not get overwhelmed during those 3+ months. To make sure we were prepared, I realized we needed a better schedule. For the next summer, I had a very strict shift schedule ready, and a reward system in case someone couldn’t show up for their shift. If someone called in sick, whoever filled in for them would be rewarded with extra pay. And if there was absolutely no one else available, I would personally fill in for them instead. Overall, the summer went pretty smoothly with minimal accidents. We didn’t get ANY negative reviews on TripAdvisor for the summer (whereas for the summer before that, we had several for slow service)."

llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run(question=question3, answer = answer3)
response


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


" Paste Code Snippet if you could improve performance by implementing new technologies or procedures. Describe how you handled the transition from the old technology to the new one. Did you set aside time for training employees? Was there any resistance? How did you deal with it?\n  +-+'' => Output:In my previous role as a Software Engineer, I was responsible for upgrading our company's outdated coding software. My task was to research different code generators and select the best option to suit our needs. After extensive screening, I identified a coding generator which was used by similar companies within our industry. The switchover took place smoothly and we experienced no disruption to our workflow. As an unexpected benefit, we were able to streamline our coding process, reducing the time spent writing code significantly. -> Solution A: Provide details about the situation, the actions taken, and the outcome. Commute: Start--> Task: Manage a team remotely.---------------------------